# Dynamic padding
To be able to group inputs of different lengths in the same batch, we need to add padding to the shortest ones, to be equal to the longedt text length.

When dealing with a whole dataset, there are various padding strategies we can apply.
## Strategies
### 1. Pad all the elements of the dataset to the same length: The length of the longest sample.
**Down sides:**
- Bathes composed from short sentences will have a lot of padding tokens, which introduce more computation in the model that it's actually not needed

### 2. Pad the elements when we batch them together, to the longest sentence inside the batch (dynamic padding)
- Batches composed of short inputs will be smaeller than the batch containing the longest sentence in the dataset.
**Down sides**
All batches have different shapes, which slows down training on othe accelerators liker TPUs

    


In [5]:
!pip install transformers evaluate datasets accelerate -q
from transformers import AutoTokenizer
from datasets import load_dataset

In [7]:
raw_datasets = load_dataset('glue','mrpc')
checkpoint = 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/home/brunojaime/Documents/Machine_learning/mlvirtualenv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:
def tokenize_function(examples):
    return tokenizer(
        examples['sentence1'],examples['sentence2'],padding="max_length",truncation=True, max_length=128 #Here we are applying max_length padding strategys
    )
tokenized_dataset = raw_datasets.map(tokenize_function,batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(['idx','sentence1','sentence2'])
tokenized_dataset = tokenized_dataset.rename_column('label','labels')
tokenized_dataset = tokenized_dataset.with_format('tensorflow')

Map: 100%|██████████| 3668/3668 [00:00<00:00, 8331.92 examples/s]


In [13]:
def tokenize_function(examples):
    return tokenizer(
        examples['sentence1'],examples['sentence2'],truncation=True #Here we are applying batch padding strategy
    )
tokenized_dataset = raw_datasets.map(tokenize_function,batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(['idx','sentence1','sentence2'])
tokenized_dataset = tokenized_dataset.rename_column('label','labels')
tokenized_dataset = tokenized_dataset.with_format('tensorflow')

Map: 100%|██████████| 1725/1725 [00:00<00:00, 8871.75 examples/s]


Dynamic batching will almost always be faster on CPUs and GPUs, so we should apply when we can. If running on TPU, or neew batches fixed shapes we should use max_length strategy